In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from python_utils import converters
import math

def get_parsed_page(url, delay=0.5):
    # This fixes a blocked by cloudflare error
    headers = {"referer": "https://www.hltv.org/betting/analytics",
               "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")


###########################################################
# COLUMN NAMES
###########################################################

column_names = ["team1", "team2", "b3651", "leovegas1", "nordic1", "unibet1", "b3652", "leovegas2", "nordic2", "unibet2", "bostyle", "team1elo", "team2elo", "team1winprob", "team1decodds", "team2winprob", "team2decodds", "teamrank1", "teamrank2", "href"]
df = pd.DataFrame(columns=column_names)

###########################################################
# HLTV ODDSPAGE
###########################################################

oddspage = get_parsed_page("http://www.hltv.org/betting/money")
upcomingmatches = oddspage.find("div", {"class": "b-match-container"})
oddsdays = oddspage.find_all("div", {"class": "b-match-container"})

for odds in oddsdays:
    oddsDetails = odds.find_all("table", {"class": "bookmakerMatch"})
    bostyle = odds.find("div", {"class": "b-best-of"}).text
    
    for getOdds in oddsDetails:
        analytics = getOdds.find("a", {"class": "a-reset"})
        
        team1 = getOdds.find_all("div", {"class": "team-name"})[0].text
        team2 = getOdds.find_all("div", {"class": "team-name"})[1].text
        
        ##################################################
        # BOOKMAKERS
        ##################################################
        
        b3651 = getOdds.find_all("td", {"class": "b-list-odds-provider-bet365"})[0].text
        b3652 = getOdds.find_all("td", {"class": "b-list-odds-provider-bet365"})[1].text
        leovegas1 = getOdds.find_all("td", {"class": "b-list-odds-provider-leovegas_se"})[0].text
        leovegas2 = getOdds.find_all("td", {"class": "b-list-odds-provider-leovegas_se"})[1].text
        nordic1 = getOdds.find_all("td", {"class": "b-list-odds-provider-betsson"})[0].text
        nordic2 = getOdds.find_all("td", {"class": "b-list-odds-provider-betsson"})[1].text
        unibet1 = getOdds.find_all("td", {"class": "b-list-odds-provider-unibet"})[0].text
        unibet2 = getOdds.find_all("td", {"class": "b-list-odds-provider-unibet"})[1].text
        
        ##################################################
        ## OPTIONAL BOOKMAKERS (for other geo locations)
        ##################################################
        
        #betway1 = getOdds.find_all("td", {"class": "b-list-odds-provider-betway"})[0].text
        #betway2 = getOdds.find_all("td", {"class": "b-list-odds-provider-betway"})[1].text
        #buff1 = getOdds.find_all("td", {"class": "b-list-odds-provider-buff88"})[0].text
        #buff2 = getOdds.find_all("td", {"class": "b-list-odds-provider-buff88"})[1].text
        #comeondk1 = getOdds.find_all("td", {"class": "b-list-odds-provider-comeondk"})[0].text
        #comeondk2 = getOdds.find_all("td", {"class": "b-list-odds-provider-comeondk"})[1].text
        #loot1 = getOdds.find_all("td", {"class": "b-list-odds-provider-lootbet"})[0].text
        #loot2 = getOdds.find_all("td", {"class": "b-list-odds-provider-lootbet"})[1].text
        #onex1 = getOdds.find_all("td", {"class": "b-list-odds-provider-onex"})[0].text
        #onex2 = getOdds.find_all("td", {"class": "b-list-odds-provider-onex"})[1].text
        #pinny1 = getOdds.find_all("td", {"class": "b-list-odds-provider-pinnacle"})[0].text
        #pinny2 = getOdds.find_all("td", {"class": "b-list-odds-provider-pinnacle"})[1].text
        #thunder1 = getOdds.find_all("td", {"class": "b-list-odds-provider-thunderpick"})[0].text
        #thunder2 = getOdds.find_all("td", {"class": "b-list-odds-provider-thunderpick"})[1].text
        
        elo1 = 0
        elo2 = 0
        teamrank1 = 50
        teamrank2 = 50
        
        ##/// Getting team ELO for calculations
        rankpage = get_parsed_page("https://www.hltv.org/ranking/teams/")
        teams = rankpage.find("div", {"class": "ranking"})
        for team in teams.find_all("div", {"class": "ranked-team standard-box"}):
            teamname = team.find('div', {"class": "ranking-header"}).select('.name')[0].text.strip()
            teamrank = converters.to_int(team.select('.position')[0].text.strip(), regexp=True)
            teampoints = converters.to_int(team.find('span', {'class': 'points'}).text, regexp=True)
            
            if teamname == team1:
                elo1 = teampoints
                teamrank1 = teamrank
                
            if teamname == team2:
                elo2 = teampoints
                teamrank2 = teamrank
                
        ##################################################
        ## CALCULATIONS MATHEMATICS FUCKKKKKKK   
        ##################################################
        
        ##/// Test data
        #elo1 = int(1500)
        #elo2 = int(1500) 
        
        
        ##/// Win probability percentages
        elo_diff = elo1 - elo2
        team1_win_prob = 1 / (1 + math.pow(10, -elo_diff/400))
        team2_win_prob = 1 / (1 + math.pow(10, elo_diff/400))
        
        ##/// Win probability odds decimals
        team1_dec_odds = 1 / team1_win_prob
        team2_dec_odds = 1 / team2_win_prob
        
        
        ##################################################
        # BUILDING THE DATAFRAME
        ##################################################
        
        ##/// Gather the data for each row
        row_data = (team1, team2, b3651, leovegas1, nordic1, unibet1, b3652, leovegas2, nordic2, unibet2, bostyle, elo1, elo2, team1_win_prob, team1_dec_odds, team2_win_prob, team2_dec_odds, teamrank1, teamrank2, 'hltv.org'+analytics['href'])

        ##/// Convert the row data to a dictionary with the column names as keys
        row_dict = dict(zip(column_names, row_data))

        ##/// Create a new DataFrame for the row
        row_df = pd.DataFrame([row_dict])

        ##/// Append the row to the DataFrame
        df = pd.concat([df, row_df])
        
        

In [11]:
df.round(2)

,team1,team2,b3651,leovegas1,nordic1,unibet1,b3652,leovegas2,nordic2,unibet2,bostyle,team1elo,team2elo,team1winprob,team1decodds,team2winprob,team2decodds,teamrank1,teamrank2,href
0,Natus Vincere,GamerLegion,1.25,1.29,1.28,1.29,3.75,3.50,3.30,3.50,BO1,806,0,0.99,1.01,0.01,104.51,3,50,hltv.org/betting/analytics/2363860/natus-vince...
0,9INE,Liquid,1.67,1.80,1.80,1.80,2.10,1.95,1.91,1.94,BO1,189,421,0.21,4.80,0.79,1.26,17,7,hltv.org/betting/analytics/2363861/9ine-vs-liq...
0,FURIA,Monte,1.67,1.65,1.68,1.65,2.10,2.15,2.06,2.15,BO1,369,144,0.79,1.27,0.21,4.65,8,22,hltv.org/betting/analytics/2363862/furia-vs-mo...
0,fnatic,Ninjas in Pyjamas,2.00,2.08,1.99,2.08,1.73,1.72,1.73,1.72,BO1,215,221,0.49,2.04,0.51,1.97,15,14,hltv.org/betting/analytics/2363863/fnatic-vs-n...
0,Heroic,FaZe,1.80,1.80,1.73,1.80,1.91,1.96,1.99,1.95,BO1,926,700,0.79,1.27,0.21,4.67,1,5,hltv.org/betting/analytics/2363864/heroic-vs-f...
0,Into the Breach,Apeks,2.50,2.50,2.34,2.50,1.50,1.50,1.53,1.50,BO1,92,120,0.46,2.17,0.54,1.85,27,24,hltv.org/betting/analytics/2363865/into-the-br...
0,Vitality,G2,1.91,2.00,1.99,2.00,1.80,1.78,1.73,1.78,BO1,893,725,0.72,1.38,0.28,3.63,2,4,hltv.org/betting/analytics/2363866/vitality-vs...
0,Bad News Eagles,ENCE,2.62,2.65,2.61,2.65,1.44,1.45,1.43,1.44,BO1,157,318,0.28,3.53,0.72,1.40,20,9,hltv.org/betting/analytics/2363867/bad-news-ea...
0,BHOP,FLUFFY AIMERS,2.25,,2.18,,1.57,,1.61,,BO3,0,0,0.50,2.00,0.50,2.00,50,50,hltv.org/betting/analytics/2364264/bhop-vs-flu...
0,MIBR,Snakes Den,1.02,1.03,1.01,1.03,10.50,9.00,11.00,9.00,BO3,76,0,0.61,1.65,0.39,2.55,30,50,hltv.org/betting/analytics/2364167/mibr-vs-sna...


In [12]:
df1 = df[['team1', 'team2','b3651', 'b3652','team1decodds', 'team2decodds', 'teamrank1','teamrank2','team1elo', 'team2elo',]]
df1.round(2)

,team1,team2,b3651,b3652,team1decodds,team2decodds,teamrank1,teamrank2,team1elo,team2elo
0,Natus Vincere,GamerLegion,1.25,3.75,1.01,104.51,3,50,806,0
0,9INE,Liquid,1.67,2.10,4.80,1.26,17,7,189,421
0,FURIA,Monte,1.67,2.10,1.27,4.65,8,22,369,144
0,fnatic,Ninjas in Pyjamas,2.00,1.73,2.04,1.97,15,14,215,221
0,Heroic,FaZe,1.80,1.91,1.27,4.67,1,5,926,700
0,Into the Breach,Apeks,2.50,1.50,2.17,1.85,27,24,92,120
0,Vitality,G2,1.91,1.80,1.38,3.63,2,4,893,725
0,Bad News Eagles,ENCE,2.62,1.44,3.53,1.40,20,9,157,318
0,BHOP,FLUFFY AIMERS,2.25,1.57,2.00,2.00,50,50,0,0
0,MIBR,Snakes Den,1.02,10.50,1.65,2.55,30,50,76,0


# csgoresults.csv

In [13]:
pd.read_csv('csgoresults.csv')

,date,team1,team2,team1score,team2score,event,bostyle,logo1,logo2,matchUrl
0,2019-07-11,Fierce,SJ,2,1,ESEA Advanced Season 31 Europe,bo3,https://static.hltv.org/images/team/logo/9397,https://static.hltv.org/images/team/logo/8504,NaN
1,2019-07-11,Lyngby Vikings,Tricked,1,2,ESEA Advanced Season 31 Europe,bo3,https://static.hltv.org/images/team/logo/8963,https://static.hltv.org/images/team/logo/4602,NaN
2,2019-07-11,Nordavind,forZe,2,1,LOOT.BET Smack My Beach Cup,bo3,https://static.hltv.org/images/team/logo/8769,https://static.hltv.org/images/team/logo/8135,NaN
3,2019-07-11,paiN,Keyd,6,16,Brasil Game Cup 2019,inf,https://static.hltv.org/images/team/logo/4773,https://static.hltv.org/images/team/logo/6033,NaN
4,2019-07-11,Sinister5,Big 5,2,1,ESEA Open Season 31 South Africa,bo3,https://static.hltv.org/images/team/logo/8730,https://static.hltv.org/images/team/logo/8111,NaN
...,...,...,...,...,...,...,...,...,...,...
16503,2023-05-07,Wildcard,Limitless,16,10,ESEA Cash Cup Circuit Season 1 Cup 6 North Ame...,ovp,https://img-cdn.hltv.org/teamlogo/3f9FTs1iRT7B...,https://img-cdn.hltv.org/teamlogo/M1NvTMcqHzhK...,/matches/2364230/wildcard-vs-limitless-esea-ca...
16504,2023-05-07,Wolsung,atputies,1,2,kleverr Kauss Spring 2023,bo3,https://img-cdn.hltv.org/teamlogo/5ofESwp8YW4H...,/img/static/team/placeholder.svg,/matches/2364243/wolsung-vs-atputies-kleverr-k...
16505,2023-05-07,Zero Tenacity,Sampi,0,2,European Pro League 2nd Division Season 3,bo3,https://img-cdn.hltv.org/teamlogo/oGL4TQTVkv21...,https://img-cdn.hltv.org/teamlogo/QPGScwyKid6-...,/matches/2364212/zero-tenacity-vs-sampi-europe...
16506,2023-05-08,ATK,Party Astronauts,0,2,ESEA Cash Cup Circuit Season 1 Cup 6 North Ame...,bo3,https://img-cdn.hltv.org/teamlogo/GNv19bHJI8jh...,https://img-cdn.hltv.org/teamlogo/5u8WkSr7yNWG...,/matches/2364239/atk-vs-party-astronauts-esea-...


# Day 1

Ninjas in Pyjamas - MOUZ <br>
1 @ 1.90 <span style= 'background:#d2f2d4'> W </span><p>
OG - ENCE <br>
2 @ 1.80 <span style= 'background:#d2f2d4'> W </span>

In [7]:
stake1 = 100
stake2 = 100

odds1 = 1.90
odds2 = 1.80

total_winnings1 = (stake1 * odds1)
total_winnings2 = (stake2 * odds2)

day1_total_stake = stake1 + stake2

day1_profit = (total_winnings1 + total_winnings2 - day1_total_stake)

day1_profit

170.0

# Day 2

Ninjas in Pyjamas - OG<br>
1 @ 1.90 <span style= 'background:#d2f2d4'> W </span><p>
forZe - Monte<br>
1 @ 2.50 <span style= 'background:#d2f2d4'> W </span>

In [8]:
stake1 = 100
stake2 = 100
odds1 = 1.90
odds2 = 2.50

total_winnings1 = (stake1 * odds1)
total_winnings2 = (stake2 * odds2)

day2_total_stake = stake1 + stake2

day2_profit = (total_winnings1 + total_winnings2 - day2_total_stake)

day2_profit

240.0

# Day 3

Grayhound - Ninjas in Pyjamas<br>
2 -1.5 @ 1.83 <span style= 'background:#d2f2d4'> W </span><p>
Apeks - paiN<br>
2 @ 1.90 <span style= 'background:#d2f2d4'> L </span>

In [13]:
stake1 = 100
stake2 = 100
odds1 = 1.83
odds2 = 1.90

total_winnings1 = (stake1 * odds1)
total_winnings2 = (stake2 * odds2)

day3_total_stake = stake1 + stake2

day3_profit = (total_winnings1 - day3_total_stake)

day3_profit

-17.0

# Total

### Profit DKK

In [14]:
total_profit = day1_profit + day2_profit + day3_profit
total_profit

393.0

### Return on investment

In [15]:
roi = (total_profit/(day1_total_stake+day2_total_stake+day3_total_stake))*100
roi

65.5